# Rete Neurale: Classificazione di fiori

Questo notebook descriverà passo passo ciò che si fa per allenare una vera e propria rete neurale (per quanto semplice). Rispetto al primo esempio, non faremo l'assunzione di dipendenza lineare e, di conseguenza, dovremo usare un approccio più generale.

Prima di tutto, descrivo qual è il problema che cerchiamo di risolvere.

## Iris Dataset

Iris è un tipo di fiore e, in base alla lunghezza del petalo e del sepalo, si può classificare in tre sottospecie. Di seguito uso Pillow e Matplotlib per visualizzare un esempio di ciascuna "classe".

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

`os` è un pacchetto che serve ad astrarre operazioni dei sistemi operativi. Per esempio su sistemi non Windows, come anche in tutti i principali linguaggi di programmazione, un percorso di un file in una cartella si scrive come `cartella/file`, mentre Windows usa `cartella\\file` (La doppia sbarra serve perché in una stringa '\\' è un carattere speciale). Usando `os` deleghiamo la gestione di operazioni specifiche a Python, e non ce ne preoccupiamo.

All'interno di questa cartella (`RetiNeurali`) ci sono una cartella `dati`, che contiene dati per i nostri problemi, e una cartella `immagini`, che contiene, be', immagini. Dunque, per aprire:

In [ ]:
percorso_immagini = os.path.join('immagini') # Equivale a .\\immagini su Windows, ./immagini su Unix.
nomi_immagini = os.listdir(percorso_immagini) # listdir crea una lista con l'elenco dei file in percorso_immagini
print(nomi_immagini)

In [ ]:
# Ora leggiamo le immagini
virginica = Image.open(os.path.join(percorso_immagini, nomi_immagini[2]))
setosa = Image.open(os.path.join(percorso_immagini, nomi_immagini[3]))
versicolor = Image.open(os.path.join(percorso_immagini, nomi_immagini[4]))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(virginica)
ax[0].set_title('Iris Virginica')
ax[1].imshow(setosa)
ax[1].set_title('Iris Setosa')
ax[2].imshow(versicolor)
ax[2].set_title('Iris Versicolor')


La specie può essere determinata misurando 4 quantità: lunghezza del sepalo, larghezza del sepalo, lunghezza del petalo e larghezza del petalo. Per fortuna qualcuno, nel 1936, ha preso le misure di 150 fiori, e i dati sono disponibili per tutti. Infatti, nella cartella `dati`, c'è un file .csv (Comma separated value), che non è altro che una lista di numeri divisi da una virgola. La prima riga può (opzionale) contenere nomi delle varie colonne. Ogni riga successiva alla prima contiene esattamente lo stesso numero di valori.  
Potremmo usare le solite funzionlità di lettura dei file di Python, ma il pacchetto `pandas` aiuta con questo tipo di file, oltre a facilitare visualizzazione ed alcune operazioni.

In [ ]:
import pandas as pd
iris = pd.read_csv(os.path.join('dati', 'iris.csv'))
iris.head() # Il metodo head() visualizza le prime n righe, default 5

La terza colonna in particolare definisce la classe della varietà. Cerchiamo di estrarre tutti i possibili valori.

In [ ]:
varieta = iris['variety']
varieta

È una lista con valori ripetuti meglio passarla a un `set` (insieme).

In [ ]:
varieta = set(varieta)
varieta

Come vedi, un `set` non contiene mai duplicati, in linea con la definizione di insieme nel senso matematico. Quindi aggiungendo un elemento che esiste già non succede niente. Nota anche le parentesi graffe (come un dizionario, ma senza coppie di valori chiave: valore)

A questo punto cerchiamo di analizzare graficamente se c'è una relazione ovvia fra le quantità. Essendo le variabili 4, le rappresento a due a due.

In [ ]:
titoli = iris.keys()[:-1] # keys dà la riga con i nomi delle colonne; variety non ci serve al momento.
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
colori = ['red','green','blue']
for riga, titolo in enumerate(titoli):
    for colonna, titolo_2 in enumerate(titoli):
        if titolo == titolo_2:
            ax[riga][colonna].text(0.5, 0.5, titolo.title(), horizontalalignment='center', fontsize=20)
        else:
            for colore, var in zip(colori, varieta):
                specie = iris[iris['variety']==var]
                ax[riga][colonna].scatter(specie[titolo], specie[titolo_2], color=colore, label=var)
            ax[riga][colonna].legend(fontsize=8)

Una relazione c'è ma non sembra esserci una legge lineare e in alcuni casi i dati non sembrano molto separabili. Cerchiamo dunque una relazione generica (non lineare), in forma di rete neurale.

### Rete Neurale

Una rete neurale è, come dice il nome, una rete di neuroni. Il concetto prende ispirazione dai neuroni naturali, in particolare, quelli che formano il cervello umano. Un neurone riceve segnali elettrici dagli altri neuroni attraverso i *dendriti*. La corrente porta cariche al neurone che si accumulano fino a una soglia limite, oltre la quale il neurone genera una corrente di *attivazione* lungo l'assione *assione*.

In [ ]:
neurone = Image.open(os.path.join(percorso_immagini, nomi_immagini[1]))
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(neurone)

Il primo modello di neurone (naturale) riportato in letteratura descriveva il funzionamento come una somma di segnali provenienti dai neuroni precedenti, passato attraverso una *funzione di attivazione*, che nel caso particolare era la funzione $\theta$:  
  
  $\theta = \begin{cases} 1,\quad \mbox{se }x \ge 0\\
                          0,\quad \mbox{altrimenti}
            \end{cases}$  
  
In pratica questo modello del neurone è molto semplice, ma ha dato modo di sviluppare modelli più complessi e, dunque, realistici. 

Da un punto di vista matematico, il concetto è però rimasto interessante, considerando che ha dato origine al fenomeno delle *reti neurali artificiali*. Scrivo quindi la prima classe neurone, per illustrare come funziona.

In [ ]:
import numpy as np

def lineare(x: float) -> float:
    return x

def theta(x: float) -> float:
    return 1 if x >= 0 else 0

class Neurone:
    def __init__(self, attivazione: str ='theta'):
        self.w = np.random.random()
        self.b = np.random.random()
        self.attivazione = theta if attivazione == 'theta' else lineare
        
    def predict(self, x: float) -> float:
        return self.attivazione(self.w*x+self.b)

Partiamo dalla classe neurone: il costruttore prende come argomento una stringa che definisce l'attivazione, e inizializza i parametri come numeri aleatori da 0 a 1 (come nell'esempio delle temperature). Inoltre, in base alla stringa, scegli la teta o niente (lineare) come funzione di attivazione (Nota: self.attivazione è una funzione). Durante la predizione, calcoliamo la *retta* e ci applichiamo la nostra funzione di attivazione. Provando ad eseguire la seguente cella più volte, il risultato cambia, ma resta sempre 0 o 1.

In [ ]:
neurone = Neurone()
neurone.predict(-0.5)

Abbiamo detto, però, che un neurone naturale prende come input vari segnali, non un solo numero. Modifichiamo dunque la classe di cui sopra per accettare un vettore di segnali.

In [ ]:
class Neurone:
    def __init__(self, dim_input, attivazione: str ='theta'):
        self.w = np.random.random((dim_input,))
        self.b = np.random.random((dim_input,))
        self.attivazione = theta if attivazione == 'theta' else lineare
        
    def predict(self, x: np.array) -> float:
        return self.attivazione(np.sum(self.w*x+self.b))

Ora la dimensione va specificata nel costruttore. Inoltre, considerando che moltiplichiamo $w$ e $b$ per l'input, la dimensione deve essere aggiustata di conseguenza. Infine, ci aspettiamo che il neurone restituisca un solo output (sempre per analogia con i neuroni naturali). Sommiamo dunque tutti i termini del vettore risultante dalla somma vettoriale e passiamo il risultato alla funzione di attivazione.  
Facciamo un esempio con `dim_input=4` (visto che il nostro dataset ha quattro variabili).

In [ ]:
neurone = Neurone(4)
neurone.predict(-1*np.ones((4,)))

In pratica, la funzione di attivazione $f(x)=\theta(x)$ non viene usato nelle reti neurali odierne. Questo principalmente perchè fa un salto repentino e la derivata è infinita in 0. In base all'applicazione, le piû frequenti sono:  
  
  $\tanh(x) = \frac{\sinh(x)}{\cosh(x)} = \frac{e^{x}-e^{-x}}{e^{x}+e^x}$
  
  $\mbox{logistic}(x)=\frac{1}{1+e^{-x}}$ 
  
  $\mbox{ReLU}(x) = \max(x, 0)$

Qui riportiamo tutta, comprese la $\theta(x)$, per completezza:

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 15))
x = np.arange(-5, 5, 0.01)
y = np.heaviside(x, 0)
ax[0][0].plot(x, y)
ax[0][0].set_title('\u03B8', fontsize='xx-large', color='lightblue')
ax[0][0].set_xlabel('x',fontsize='xx-large', color='lightblue')
ax[0][0].set_ylabel('y',fontsize='xx-large', color='lightblue')
y = np.tanh(x)
ax[0][1].plot(x, y)
ax[0][1].set_title('tanh', fontsize='xx-large', color='lightblue')
ax[0][1].set_xlabel('x', fontsize='xx-large', color='lightblue')
ax[0][1].set_ylabel('y', fontsize='xx-large', color='lightblue')
y = 1./(1+np.exp(-x))
ax[1][0].plot(x, y)
ax[1][0].set_title('logistic', fontsize='xx-large', color='lightblue')
ax[1][0].set_xlabel('x',fontsize='xx-large', color='lightblue')
ax[1][0].set_ylabel('y',fontsize='xx-large', color='lightblue')
y = np.maximum(x, np.zeros_like(x))
ax[1][1].plot(x, y)
ax[1][1].set_title('ReLU', fontsize='xx-large', color='lightblue')
ax[1][1].set_xlabel('x', fontsize='xx-large', color='lightblue')
ax[1][1].set_ylabel('y', fontsize='xx-large', color='lightblue')

In questo esempio continuerò a usare la logistica, perchè ha una derivata continua ma, al contrario della tanh, più semplice da calcolare (non solo per noi, ma anche per il pc). 

A questo punto abbiamo tutti gli elementi per una rete neurale, eccetto che un neurone singolo non fa proprio da rete neurale. Infatti, un rete è composta da una serie di neuroni, come nell'esempio:

In [ ]:
def logistic(x: float)-> float:
    return 1./(1.+np.exp(-x))

class Neurone:
    def __init__(self, dim_input, attivazione: str ='logistic'):
        self.w = np.random.random((dim_input,))
        self.b = np.random.random((dim_input,))
        self.attivazione = logistic if attivazione == 'logistic' else lineare
        
    def predict(self, x: np.array) -> float:
        return self.attivazione(np.sum(self.w*x+self.b))

In [ ]:
class Layer:
    def __init__(self, n_input: int, n_output: int, attivazione: str = 'logistic'):
        self.neurons = [Neurone(n_input, attivazione) for _ in range(n_output)]
        
    def predict(self, x: np.array) -> np.array:
        y = np.array([neuro.predict(x) for neuro in self.neurons])
        return y

In [ ]:
class ReteNeurale:
    def __init__(self, n_input: int, n_output: int = 1):
        # IMPORTANTE!!! L'output di un layer deve essere uguale all'input del successivo
        self.layers = [Layer(n_input, 5), Layer(5, n_output, '')]
    
    def predict(self, x: np.array) -> np.array:
        y = x 
        for layer in self.layers:
            y = layer.predict(y)
        return y

In [ ]:
network = ReteNeurale(4, )
network.predict(np.random.random((4,)))

Questo è un esempio completo del passaggio in avanti (*forward step*) di un rete neurale. Cerchiamo di capire cosa succede:
- La classe `ReteNeurale` costruisce l'architettura della rete a partire di *strati* (layers) di neuroni. Nell'esempio ci sono due strati: Il primo prende 4 input (determinato dai dati), e resitutisce 5 output, mentre il secondo prende quei 5 input e restituisce 1 output, un numero che dovrà essere "vicino" a 0, 1, 2 (le classi del dataset).
- La classe Layer raggruppa tutti i neuroni che prendono lo stesso input, e restituisce tanti output quanto sono i neuroni di cui è composto. Inoltre specifica l'attivazione che ogni strato di neuroni deve usare (neuroni nello stesso strato hanno tutti la stessa).
- Il neurone è analogo a quello precedente, ma uso l'attivazione logistica, come detto prima.

La figura in basso rappresenta lo schema della rete definita sopra.

In [ ]:
neurone = Image.open(os.path.join(percorso_immagini, nomi_immagini[0]))
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(neurone)

Ricordiamoci ora cosa succedeva nell'esempio della retta: i dati venivano passati alla "rete" (la nostra funzione retta), con i parametri inizializzati a caso. Questi andavano poi ottimizzati passando tutti i possibili valori, varie volte o *epoche*. In questo esempio, l'approccio è lo stesso ma ovviamente il calcolo è più complesso.